# 使用 Meta 家族模型进行开发

## 简介

本课将涵盖：

- 探索 Meta 家族的两大主力模型——Llama 3.1 和 Llama 3.2
- 了解每个模型的应用场景和适用案例
- 通过代码示例展示每个模型的独特功能

## Meta 家族模型

在本课中，我们将探索 Meta 家族（也称为 “Llama Herd”）中的两个模型——Llama 3.1 和 Llama 3.2

这些模型有不同的版本，并且可以在 Github Model 市场中获取。更多关于如何使用 Github Models 进行 [AI 模型原型开发](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) 的信息可以参考这里。

模型版本：
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*注意：Llama 3 也可以在 Github Models 上获取，但本课不涉及该模型*


## Llama 3.1

拥有4050亿参数，Llama 3.1属于开源大语言模型（LLM）范畴。

这个版本是在之前发布的Llama 3基础上的升级，主要包括：

- 更大的上下文窗口——128k tokens，相比之前的8k tokens
- 更高的最大输出tokens——4096，相比之前的2048
- 更强的多语言支持——因为训练tokens数量增加

这些提升让Llama 3.1在构建生成式AI应用时能够应对更复杂的场景，包括：
- 原生函数调用——可以在LLM流程之外调用外部工具和函数
- 更强的RAG性能——得益于更大的上下文窗口
- 合成数据生成——能够为微调等任务高效生成数据


### 原生函数调用

Llama 3.1 已经过微调，现在在调用函数或工具时更加高效。它还内置了两个工具，模型可以根据用户的提示自动判断是否需要使用。这两个工具是：

- **Brave Search** - 可以通过网络搜索获取最新的信息，比如天气情况
- **Wolfram Alpha** - 可以用于更复杂的数学计算，因此你无需自己编写相关函数

你也可以创建自己的自定义工具供 LLM 调用。

在下面的代码示例中：

- 我们在系统提示中定义了可用的工具（brave_search, wolfram_alpha）
- 发送一个用户提示，询问某个城市的天气
- LLM 会用工具调用的方式回复，调用 Brave Search 工具，格式如下：`<|python_tag|>brave_search.call(query="Stockholm weather")`

*注意：这个示例只展示了工具的调用。如果你想获取实际结果，需要在 Brave API 页面注册一个免费账号，并自行定义该函数*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

尽管 Llama 3.1 是一个大语言模型，但它的一个局限性在于多模态能力。也就是说，它无法像 Llama 3.2 一样，使用不同类型的输入（比如图片）作为提示并给出回应。这项能力正是 Llama 3.2 的主要特性之一。其他新特性还包括：

- 多模态能力 —— 能够同时处理文本和图片提示
- 小到中等规模的模型（11B 和 90B）—— 提供灵活的部署选择
- 仅文本版本（1B 和 3B）—— 适合在边缘设备或移动设备上部署，延迟更低

多模态支持是开源模型领域的一大进步。下面的代码示例展示了如何同时输入图片和文本提示，让 Llama 3.2 90B 对图片进行分析。

### Llama 3.2 的多模态支持


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## 学习不止于此，继续你的旅程

完成本课后，欢迎访问我们的[生成式 AI 学习合集](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst)，持续提升你的生成式 AI 知识！



---

**免责声明**：  
本文件由 AI 翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻译。我们力求准确，但请注意，自动翻译可能包含错误或不准确之处。原始语言版本应被视为权威来源。对于关键信息，建议使用专业人工翻译。因使用本翻译而产生的任何误解或曲解，我们概不负责。
